In [0]:
-- Cria o SCHEMA gold dentro do CATÁLOGO 'jobs'
CREATE SCHEMA IF NOT EXISTS jobs.gold;

-- Dim_Habilidade: chave hash única para desnormalizar as habilidades
CREATE OR REPLACE TABLE jobs.gold.dim_habilidade AS
SELECT
    -- Chave Técnica para garantir unicidade da habilidade
    SHA2(
        CONCAT_WS('|', SKILL_NAME, SKILL_CATEGORY_NAME, SKILL_SUBCATEGORY_NAME, CAST(IS_SOFTWARE AS STRING)),
        256
    ) AS ID_HABILIDADE_HASH, 
    SKILL_NAME,
    SKILL_CATEGORY_NAME,
    SKILL_SUBCATEGORY_NAME,
    IS_SOFTWARE 
FROM
    jobs.silver.vagas_skills_limpas 
GROUP BY ALL;

-- Fato_Demanda (Tabela Fato): Vincula Vaga (ID) e Habilidade (ID_HABILIDADE_HASH)
CREATE OR REPLACE TABLE jobs.gold.fato_demanda AS
SELECT DISTINCT
    T1.ID, 
    T2.ID_HABILIDADE_HASH
FROM
    jobs.silver.vagas_skills_limpas T1
INNER JOIN
    jobs.gold.dim_habilidade T2
ON
    T1.SKILL_NAME = T2.SKILL_NAME
    AND T1.SKILL_CATEGORY_NAME = T2.SKILL_CATEGORY_NAME;

SELECT 'Camada GOLD (Esquema Estrela) criada com sucesso.' AS Status;